In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

l = [369, 742, 1680, 140, 582, 1745, 805, 3369, 3620, 3529, 3165, 1888, 1420, 547, 772, 1,852, 860, 545, 804, 1847, 544, 1458, 834, 1998, 83, 664, 911, 1534, 1142, 3388, 2757, 1584, 2484, 3482, 1658, 1046, 2989, 2070, 160, 118, 743]

def get_party(s_no):
    if 1 <= s_no <= 240:
        return 'BJP'
    elif 241 <= s_no <= 339:
        return 'INC'
    elif 340 <= s_no <= 376:
        return 'SP'
    elif 377 <= s_no <= 405:
        return 'AITC'
    elif 406 <= s_no <= 427:
        return 'DMK'
    elif 428 <= s_no <= 443:
        return 'TDP'
    elif 444 <= s_no <= 455:
        return 'JD'
    elif 456 <= s_no <= 464:
        return 'SHSUBT'
    elif 465 <= s_no <= 472:
        return 'NCPSP'
    elif 473 <= s_no <= 479:
        return 'SHS'
    elif 480 <= s_no <= 484:
        return 'LJPRV'
    elif 485 <= s_no <= 488:
        return 'YSRCP'
    elif 489 <= s_no <= 492:
        return 'RJD'
    elif 493 <= s_no <= 496:
        return 'CPI(M)'
    elif 497 <= s_no <= 499:
        return 'IUML'
    elif 500 <= s_no <= 502:
        return 'AAAP'
    elif 503 <= s_no <= 505:
        return 'JMM'
    elif 506 <= s_no <= 507:
        return 'JnP'
    elif 508 <= s_no <= 509:
        return 'CPI(ML)(L)'
    elif 510 <= s_no <= 511:
        return 'JD(S)'
    elif 512 <= s_no <= 513:
        return 'VCK'
    elif 514 <= s_no <= 515:
        return 'CPI'
    elif 516 <= s_no <= 517:
        return 'RLD'
    elif 518 <= s_no <= 519:
        return 'JKN'
    elif 520 == s_no:
        return 'UPPL'
    elif 521 == s_no:
        return 'AGP'
    elif 522 == s_no:
        return 'HAMS'
    elif 523 == s_no:
        return 'KEC'
    elif 524 == s_no:
        return 'RSP'
    elif 525 == s_no:
        return 'NCP'
    elif 526 == s_no:
        return 'VOTPP'
    elif 527 == s_no:
        return 'ZPM'
    elif 528 == s_no:
        return 'SAD'
    elif 529 == s_no:
        return 'RLTP'
    elif 530 == s_no:
        return 'BHRTADVSIP'
    elif 531 == s_no:
        return 'SKM'
    elif 532 == s_no:
        return 'MDMK'
    elif 533 == s_no:
        return 'ASPKR'
    elif 534 == s_no:
        return 'ADAL'
    elif 535 == s_no:
        return 'AJSUP'
    elif 536 == s_no:
        return 'AIMIM'
    elif 537 <= s_no <= 543:
        return 'IND(Independent)'
    else:
        return 'Unknown'

all_results_df = pd.DataFrame()

serial_number = 1

for i in l:
    url = f"https://results.eci.gov.in/PcResultGenJune2024/partywisewinresultState-{i}.htm"

    response = requests.get(url)
    response.encoding = 'utf-8'

    soup = BeautifulSoup(response.text, 'html.parser')

    results_div = soup.find('div', {'class': 'table-responsive'})

    if results_div:
        s_no = []
        parliament_constituency = []
        winning_candidate = []
        total_votes = []
        margin = []

        for row in results_div.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) >= 5:
                s_no.append(serial_number)
                parliament_constituency.append(cells[1].text.strip())
                winning_candidate.append(cells[2].text.strip())
                total_votes.append(cells[3].text.strip())
                margin.append(cells[4].text.strip())
                serial_number += 1

        results_df = pd.DataFrame({
            'S.No': s_no,
            'Parliament Constituency': parliament_constituency,
            'Winning Candidate': winning_candidate,
            'Total Votes': total_votes,
            'Margin': margin
        })

        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

    else:
        print(f"Could not find the results div on the page for URL with identifier {i}. Please check the HTML structure.")

all_results_df['Party'] = all_results_df['S.No'].apply(get_party)

all_results_df.to_csv('final_election_results.csv', index=False)
print("Data has been scraped and saved to 'final_election_results.csv'.")


Data has been scraped and saved to 'lok_sabha_election_results_2024.csv'.


In [18]:
import pandas as pd

df = pd.read_csv('final_election_results.csv')
pd.set_option('expand_frame_repr', False)

df['Total Votes'] = pd.to_numeric(df['Total Votes'], errors='coerce')

df['Margin'] = pd.to_numeric(df['Margin'], errors='coerce')

highest_votes = df[df['Total Votes'] == df['Total Votes'].max()]
print("1. Parliament Constituency with the highest total votes:")
print(highest_votes)

lowest_margin = df[df['Margin'] == df['Margin'].min()]
print("\n2. Parliament Constituency with the lowest margin of victory:")
print(lowest_margin)

largest_margin = df[df['Margin'] == df['Margin'].max()]
print("\n3. Parliament Constituency with the largest margin of victory:")
print(largest_margin)

total_votes_by_party = df.groupby('Party')['Total Votes'].sum().sort_values(ascending=False).head(1)
print("\n4. Party with the highest total votes:")
print(total_votes_by_party)

top_5_candidates = df.nlargest(5, 'Total Votes')
print("\n5. Top 5 winning candidates by total votes:")
print(top_5_candidates[['Winning Candidate', 'Parliament Constituency', 'Total Votes']])

top_5_turnout = df.nlargest(5, 'Total Votes')
print("\n6. Top 5 constituencies with highest voter turnout:")
print(top_5_turnout[['Parliament Constituency', 'Total Votes']])

average_margin_party = df.groupby('Party')['Margin'].mean().sort_values(ascending=False)
print("\n7. Average margin of victory by party:")
print(average_margin_party)

close_contests = df[df['Margin'] < 5000]
num_close_contests = close_contests.shape[0]
print(f"\n8. Number of close contests (margin < 5000 votes): {num_close_contests}")

average_votes_party = df.groupby('Party')['Total Votes'].mean().sort_values(ascending=False)
print("\n9. Average total votes by party:")
print(average_votes_party)

least_votes_winners = df.nsmallest(5, 'Total Votes')
print("\n10. Winning candidates with the least total votes:")
print(least_votes_winners[['Winning Candidate', 'Parliament Constituency', 'Total Votes']])


1. Parliament Constituency with the highest total votes:
     S.No Parliament Constituency Winning Candidate  Total Votes     Margin Party
240   241              Dhubri (2)   RAKIBUL HUSSAIN    1471885.0  1012476.0   INC

2. Parliament Constituency with the lowest margin of victory:
     S.No Parliament Constituency         Winning Candidate  Total Votes  Margin Party
476   477   Mumbai North West(27)  RAVINDRA DATTARAM WAIKAR     452644.0    48.0   SHS

3. Parliament Constituency with the largest margin of victory:
     S.No Parliament Constituency Winning Candidate  Total Votes     Margin Party
104   105              INDORE(26)   SHANKAR LALWANI    1226751.0  1175092.0   BJP

4. Party with the highest total votes:
Party
BJP    157039540.0
Name: Total Votes, dtype: float64

5. Top 5 winning candidates by total votes:
         Winning Candidate Parliament Constituency  Total Votes
240        RAKIBUL HUSSAIN              Dhubri (2)    1471885.0
104        SHANKAR LALWANI              IN